# NBA Production Predictor - AI Model
This program is designed to take in 5, 7, or 9 players on each team and compare them based on historical stats and our analysis on player production. Essentially the model will determine which team is stronger based past production. Our hopes is that the model can take into account how each player performed against certaint teams and other players in their perspective position, which may be also be a future goal of our model. Thank you for viewing and I hope you enjoy using our NBA Game Predictor Model. 

## Import project dependencies

In [2]:
# Import pandas to read and manipulate data
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from utils import *
import os
from dotenv import load_dotenv

In [3]:
# Using pandas read in each csv project file
regular_season_data = pd.read_csv('../data/Regular_Season.csv')
playoff_data = pd.read_csv('../data/Playoffs.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/Regular_Season.csv'

In [ ]:
# View the first five columns of each file to ensure they were read in correctly
display(regular_season_data.head())
display(playoff_data.tail())

,Unnamed: 0.1,Unnamed: 0,year,Season_type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,AST_TOV,STL_TOV
0,0,0,2012-13,Regular_Season,201142,1,Kevin Durant,1610612760,OKC,81,...,594,640,374,116,105,280,143,2280,1.34,0.41
1,1,1,2012-13,Regular_Season,977,2,Kobe Bryant,1610612747,LAL,78,...,367,433,469,106,25,287,173,2133,1.63,0.37
2,2,2,2012-13,Regular_Season,2544,3,LeBron James,1610612748,MIA,76,...,513,610,551,129,67,226,110,2036,2.44,0.57
3,3,3,2012-13,Regular_Season,201935,4,James Harden,1610612745,HOU,78,...,317,379,455,142,38,295,178,2023,1.54,0.48
4,4,4,2012-13,Regular_Season,2546,5,Carmelo Anthony,1610612752,NYK,67,...,326,460,171,52,32,175,205,1920,0.98,0.30


,Unnamed: 0.1,Unnamed: 0,year,Season_type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,AST_TOV,STL_TOV
0,0,468,2012-13,Playoffs,2544,1,LeBron James,1610612748,MIA,23,...,156,193,152,41,18,70,43,596,2.17,0.59
1,1,469,2012-13,Playoffs,2225,2,Tony Parker,1610612759,SAS,21,...,54,68,146,24,3,53,26,432,2.75,0.45
2,2,470,2012-13,Playoffs,1495,3,Tim Duncan,1610612759,SAS,21,...,160,214,40,18,34,42,53,381,0.95,0.43
3,3,471,2012-13,Playoffs,202331,4,Paul George,1610612754,IND,19,...,125,141,96,25,9,75,72,365,1.28,0.33
4,4,472,2012-13,Playoffs,2548,5,Dwyane Wade,1610612748,MIA,22,...,64,102,105,38,23,58,45,349,1.81,0.66


# Beginning of Preprocessing PipeLine Utils.py
1. combine_dataframes
    - Uses reformat_columns and converts 'Season_type' and 'year' columns
3. encode_categorical_columns
4. convert_to_single_game
5. calculate_shooting_production
6. calculate_offensive_anx
7. calculate_defensive_anx
8. calculate_total_player_production

### 1. Combine the two dataframes 'regular_season_data' and 'playoff_data'

In [ ]:
"""
Function Name: combine_dataframes

Parameters: Takes in a [list of dataframes]

Output: A combined dataframe

Description: This function takes a list of DataFrames, reformats each, and 
combines them into one DataFrame along axis 0.

"""
# Put dataframes into a list
list_of_dataframes = [regular_season_data, playoff_data]

# Call 'combine_dataframes' function to combine the list of dataframes
combined_stats = combine_dataframes(list_of_dataframes)
combined_stats.head()

,year,Season_type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,AST_TOV,STL_TOV
0,2012,Regular Season,201142,1,Kevin Durant,1610612760,OKC,81,3119,731,...,594,640,374,116,105,280,143,2280,1.34,0.41
1,2012,Regular Season,977,2,Kobe Bryant,1610612747,LAL,78,3013,738,...,367,433,469,106,25,287,173,2133,1.63,0.37
2,2012,Regular Season,2544,3,LeBron James,1610612748,MIA,76,2877,765,...,513,610,551,129,67,226,110,2036,2.44,0.57
3,2012,Regular Season,201935,4,James Harden,1610612745,HOU,78,2985,585,...,317,379,455,142,38,295,178,2023,1.54,0.48
4,2012,Regular Season,2546,5,Carmelo Anthony,1610612752,NYK,67,2482,669,...,326,460,171,52,32,175,205,1920,0.98,0.30


In [ ]:
combined_stats['PLAYER'].unique()

NameError: name 'combined_stats' is not defined

### 2. Encode categorical columns

In [ ]:
"""
Function Name: encode_categorical_columns

Parameters: Takes in a dataframe

Output: Converted dataframe, with dropped categorical columns, along with player and team mappings. 
* Converted Dataframe
The player and team columns have been encoded and mapped so they can be decoded later.
* Player and Team Mappings
The function returns the player mappings and the team mappings so that they can be decoded later.

Description: Encodes categorical columns, like Season_type, PLAYER, and TEAM, using 
LabelEncoder. Also, it returns mappings for decoding later.

"""

# Call the 'convert_to_single_game' function to convert dataframe
encoded_df, player_mapping, team_mapping = encode_categorical_columns(combined_stats)
encoded_df.head()

,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,STL,BLK,TOV,PF,PTS,AST_TOV,STL_TOV,season_type_encoded,player_encoded,team_encoded
0,81,3119,731,1433,0.510,139,334,0.416,679,750,...,116,105,280,143,2280,1.34,0.41,0,895,21
1,78,3013,738,1595,0.463,132,407,0.324,525,626,...,106,25,287,173,2133,1.63,0.37,0,922,13
2,76,2877,765,1354,0.565,103,254,0.406,403,535,...,129,67,226,110,2036,2.44,0.57,0,963,15
3,78,2985,585,1337,0.438,179,486,0.368,674,792,...,142,38,295,178,2023,1.54,0.48,0,653,10
4,67,2482,669,1489,0.449,157,414,0.379,425,512,...,52,32,175,205,1920,0.98,0.30,0,198,20


### 3. Convert dataframe to single game stats

In [ ]:
"""
Function Name: convert_to_single_game

Parameters: A dataframe

Output: Converted dataframe

Description: This function converts per-season stats to per-game stats by dividing each column by the games played (GP) column, while excluding certain 
columns (like percentage columns) from the division.

"""

single_game_stats = convert_to_single_game(encoded_df)
single_game_stats.head()

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,STL,BLK,TOV,PF,PTS,AST_TOV,STL_TOV,season_type_encoded,player_encoded,team_encoded
0,38.5,9.0,17.7,0.510,1.7,4.1,0.416,8.4,9.3,0.905,...,1.4,1.3,3.5,1.8,28.1,0.0,0.0,0.0,11.0,0.3
1,38.6,9.5,20.4,0.463,1.7,5.2,0.324,6.7,8.0,0.839,...,1.4,0.3,3.7,2.2,27.3,0.0,0.0,0.0,11.8,0.2
2,37.9,10.1,17.8,0.565,1.4,3.3,0.406,5.3,7.0,0.753,...,1.7,0.9,3.0,1.4,26.8,0.0,0.0,0.0,12.7,0.2
3,38.3,7.5,17.1,0.438,2.3,6.2,0.368,8.6,10.2,0.851,...,1.8,0.5,3.8,2.3,25.9,0.0,0.0,0.0,8.4,0.1
4,37.0,10.0,22.2,0.449,2.3,6.2,0.379,6.3,7.6,0.830,...,0.8,0.5,2.6,3.1,28.7,0.0,0.0,0.0,3.0,0.3


### 4. Calculate Shooting Production

In [ ]:
"""
Function Name: calculate_shooting_production

Parameters: A dataframe - stats

Output: Converted dataframe

Description: This function calculates shooting production by multiplying made shots 
by weights defined in the .env file for two-pointers, three-pointers, and free throws.

"""

# Call the function on the single stats dataframe
shooting_production_calculated = calculate_shooting_production(single_game_stats)
shooting_production_calculated.head()

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,season_type_encoded,player_encoded,team_encoded,FGMI,FG3MI,FTMI,two_production,three_production,ft_production,total_shooting_production
0,38.5,9.0,17.7,0.510,1.7,4.1,0.416,8.4,9.3,0.905,...,0.0,11.0,0.3,8.7,2.4,0.9,35.4,12.3,7.5,55.2
1,38.6,9.5,20.4,0.463,1.7,5.2,0.324,6.7,8.0,0.839,...,0.0,11.8,0.2,10.9,3.5,1.3,40.8,15.6,5.4,61.8
2,37.9,10.1,17.8,0.565,1.4,3.3,0.406,5.3,7.0,0.753,...,0.0,12.7,0.2,7.7,1.9,1.7,35.6,9.9,3.6,49.1
3,38.3,7.5,17.1,0.438,2.3,6.2,0.368,8.6,10.2,0.851,...,0.0,8.4,0.1,9.6,3.9,1.6,34.2,18.6,7.0,59.8
4,37.0,10.0,22.2,0.449,2.3,6.2,0.379,6.3,7.6,0.830,...,0.0,3.0,0.3,12.2,3.9,1.3,44.4,18.6,5.0,68.0


### 5. Calculate Offensive Anx

In [ ]:
"""
Function Name: calculate_offensive_anx

Parameters: A dataframe - stats

Output: Converted dataframe

Description: This function calculates offensive anxiety by applying weights
from environment variables to specific offensive stats.

"""

# Call the function to calculate offensive anx
offensive_anx_calculated = calculate_offensive_anx(shooting_production_calculated)
offensive_anx_calculated.head()

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,FG3MI,FTMI,two_production,three_production,ft_production,total_shooting_production,weighted_or,weighted_ast,weighted_to,total_offensive_anx
0,38.5,9.0,17.7,0.510,1.7,4.1,0.416,8.4,9.3,0.905,...,2.4,0.9,35.4,12.3,7.5,55.2,0.30,1.15,-1.75,-0.30
1,38.6,9.5,20.4,0.463,1.7,5.2,0.324,6.7,8.0,0.839,...,3.5,1.3,40.8,15.6,5.4,61.8,0.40,1.50,-1.85,0.05
2,37.9,10.1,17.8,0.565,1.4,3.3,0.406,5.3,7.0,0.753,...,1.9,1.7,35.6,9.9,3.6,49.1,0.65,1.80,-1.50,0.95
3,38.3,7.5,17.1,0.438,2.3,6.2,0.368,8.6,10.2,0.851,...,3.9,1.6,34.2,18.6,7.0,59.8,0.40,1.45,-1.90,-0.05
4,37.0,10.0,22.2,0.449,2.3,6.2,0.379,6.3,7.6,0.830,...,3.9,1.3,44.4,18.6,5.0,68.0,1.00,0.65,-1.30,0.35


### 6. Calculate Defensive Anx  

In [ ]:
"""
Function Name: calculate_defensive_anx 

Parameters: Takes in a dataframe

Output: Converted dataframe

Description: This function calculates defensive anxiety by applying weights from 
environment variables to specific defensive stats.

"""

# Call the 'reformat_columns' function to convert dataframe
defensive_anx_calculated = calculate_defensive_anx(offensive_anx_calculated)
defensive_anx_calculated.head()

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,total_shooting_production,weighted_or,weighted_ast,weighted_to,total_offensive_anx,weighted_dr,weighted_stl,weighted_blk,weighted_pf,total_defensive_anx
0,38.5,9.0,17.7,0.510,1.7,4.1,0.416,8.4,9.3,0.905,...,55.2,0.30,1.15,-1.75,-0.30,1.825,0.350,0.65,0.450,3.275
1,38.6,9.5,20.4,0.463,1.7,5.2,0.324,6.7,8.0,0.839,...,61.8,0.40,1.50,-1.85,0.05,1.175,0.350,0.15,0.550,2.225
2,37.9,10.1,17.8,0.565,1.4,3.3,0.406,5.3,7.0,0.753,...,49.1,0.65,1.80,-1.50,0.95,1.700,0.425,0.45,0.350,2.925
3,38.3,7.5,17.1,0.438,2.3,6.2,0.368,8.6,10.2,0.851,...,59.8,0.40,1.45,-1.90,-0.05,1.025,0.450,0.25,0.575,2.300
4,37.0,10.0,22.2,0.449,2.3,6.2,0.379,6.3,7.6,0.830,...,68.0,1.00,0.65,-1.30,0.35,1.225,0.200,0.25,0.775,2.450


### 7. Calculate Total Player Production

In [ ]:
"""
Function Name: calculate_total_player_production

Parameters: Takes in a [list of dataframes]

Output: A combined dataframe

Description: This function calculates total player production by summing the 
shooting production, offensive anxiety, and defensive anxiety.

"""

# Call a function used to calculate the total production column
total_production_calculated = calculate_total_player_production(defensive_anx_calculated)
total_production_calculated.head()

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,weighted_or,weighted_ast,weighted_to,total_offensive_anx,weighted_dr,weighted_stl,weighted_blk,weighted_pf,total_defensive_anx,total_production
0,38.5,9.0,17.7,0.510,1.7,4.1,0.416,8.4,9.3,0.905,...,0.30,1.15,-1.75,-0.30,1.825,0.350,0.65,0.450,3.275,58.175
1,38.6,9.5,20.4,0.463,1.7,5.2,0.324,6.7,8.0,0.839,...,0.40,1.50,-1.85,0.05,1.175,0.350,0.15,0.550,2.225,64.075
2,37.9,10.1,17.8,0.565,1.4,3.3,0.406,5.3,7.0,0.753,...,0.65,1.80,-1.50,0.95,1.700,0.425,0.45,0.350,2.925,52.975
3,38.3,7.5,17.1,0.438,2.3,6.2,0.368,8.6,10.2,0.851,...,0.40,1.45,-1.90,-0.05,1.025,0.450,0.25,0.575,2.300,62.050
4,37.0,10.0,22.2,0.449,2.3,6.2,0.379,6.3,7.6,0.830,...,1.00,0.65,-1.30,0.35,1.225,0.200,0.25,0.775,2.450,70.800


####